# Logging results and uploading models to Comet ML
In this example, we train a simple XGBoost model and log the training
results to Comet ML. We also save the resulting model checkpoints
as artifacts.

Let's start with installing our dependencies:

In [1]:
!pip install -qU "ray[tune]" sklearn xgboost_ray comet_ml

Then we need some imports:

In [2]:
import ray

from ray.air.config import RunConfig, ScalingConfig
from ray.air.result import Result
from ray.train.xgboost import XGBoostTrainer
from ray.air.integrations.comet import CometLoggerCallback

We define a simple function that returns our training dataset as a Ray Dataset:

In [3]:
def get_train_dataset() -> ray.data.Dataset:
    dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")
    return dataset

Now we define a simple training function. All the magic happens within the `CometLoggerCallback`:

```python
CometLoggerCallback(
    project_name=comet_project,
    save_checkpoints=True,
)
```

It will automatically log all results to Comet ML and upload the checkpoints as artifacts. It assumes you're logged in into Comet via an API key or your `~./.comet.config`.

In [4]:
def train_model(train_dataset: ray.data.Dataset, comet_project: str) -> Result:
    """Train a simple XGBoost model and return the result."""
    trainer = XGBoostTrainer(
        scaling_config=ScalingConfig(num_workers=2),
        params={"tree_method": "auto"},
        label_column="target",
        datasets={"train": train_dataset},
        num_boost_round=10,
        run_config=RunConfig(
            callbacks=[
                # This is the part needed to enable logging to Comet ML.
                # It assumes Comet ML can find a valid API (e.g. by setting
                # the ``COMET_API_KEY`` environment variable).
                CometLoggerCallback(
                    project_name=comet_project,
                    save_checkpoints=True,
                )
            ]
        ),
    )
    result = trainer.fit()
    return result

Let's kick off a run:

In [5]:
comet_project = "ray_air_example"

train_dataset = get_train_dataset()
result = train_model(train_dataset=train_dataset, comet_project=comet_project)

2022-05-19 15:19:17,237	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc,iter,total time (s),train-rmse
XGBoostTrainer_ac544_00000,TERMINATED,127.0.0.1:19852,10,9.7203,0.030717


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
(raylet) 2022-05-19 15:19:21,584	INFO context.py:70 -- Exec'ing worker with command: exec /Users/kai/.pyenv/versions/3.7.7/bin/python3.7 /Users/kai/coding/ray/python/ray/workers/default_worker.py --node-ip-address=127.0.0.1 --node-manager-port=61222 --object-store-name=/tmp/ray/session_2022-05-19_15-19-14_632568_19778/sockets/plasma_store --raylet-name=/tmp/ray/session_2022-05-19_15-19-14_632568_19778/sockets/raylet --redis-address=None --storage=None --temp-dir=/tmp/ray --metrics-agent-port=62873 --logging-rotate-bytes=536870912 --logging-rotate-backup-count=5 --gcs-address=127.0.0.1:61938 --redis-password=5241590000000000 --startup-token=16 --runtime-env-hash=-2010331134
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/krfricke/ray-air-example/ecd3726ca127497ba7386003a249fad6

COMET WARNING: Failed 

Result for XGBoostTrainer_ac544_00000:
  date: 2022-05-19_15-19-30
  done: false
  experiment_id: d3007bd6a2734b328fd90385485c5a8d
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 19852
  should_checkpoint: true
  time_since_restore: 6.529659032821655
  time_this_iter_s: 6.529659032821655
  time_total_s: 6.529659032821655
  timestamp: 1652969970
  timesteps_since_restore: 0
  train-rmse: 0.357284
  training_iteration: 1
  trial_id: ac544_00000
  warmup_time: 0.003961086273193359
  


COMET INFO: Scheduling the upload of 3 assets for a size of 2.48 KB, this can take some time
COMET INFO: Artifact 'krfricke/checkpoint_XGBoostTrainer_ac544_00000:1.0.0' has started uploading asynchronously
COMET WARNING: The given value of the metric episodes_total was None; ignoring
COMET WARNING: The given value of the metric timesteps_total was None; ignoring
COMET INFO: Artifact 'checkpoint_XGBoostTrainer_ac544_00000' version 2.0.0 created (previous was: 1.0.0)
COMET INFO: Scheduling the upload of 3 assets for a size of 3.86 KB, this can take some time
COMET INFO: Artifact 'krfricke/checkpoint_XGBoostTrainer_ac544_00000:2.0.0' has started uploading asynchronously
COMET WARNING: The given value of the metric episodes_total was None; ignoring
COMET WARNING: The given value of the metric timesteps_total was None; ignoring
COMET INFO: Artifact 'checkpoint_XGBoostTrainer_ac544_00000' version 3.0.0 created (previous was: 2.0.0)
COMET INFO: Artifact 'krfricke/checkpoint_XGBoostTrainer_ac5

Result for XGBoostTrainer_ac544_00000:
  date: 2022-05-19_15-19-33
  done: true
  experiment_id: d3007bd6a2734b328fd90385485c5a8d
  experiment_tag: '0'
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 19852
  should_checkpoint: true
  time_since_restore: 9.720295906066895
  time_this_iter_s: 0.39761900901794434
  time_total_s: 9.720295906066895
  timestamp: 1652969973
  timesteps_since_restore: 0
  train-rmse: 0.030717
  training_iteration: 10
  trial_id: ac544_00000
  warmup_time: 0.003961086273193359
  


2022-05-19 15:19:35,621	INFO tune.py:753 -- Total run time: 15.75 seconds (14.94 seconds for the tuning loop).


Check out your [Comet ML](https://www.comet.ml/) project to see the results!